In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline


In [2]:
# df = pd.read_csv('../data/analyst_ratings_processed_final.csv')[['headline', 'stock', 'headline_cleaned']]
df = pd.read_csv('../data/analyst_ratings_processed_final.csv')
# df = df.sample(50000)


In [3]:
def stockDevelopement(row):
    stock_differenz = row['open'] - row['close']
    # return 1 - row['close']/row['open']
    if stock_differenz < 1:
        return -1
    elif stock_differenz > 1:
        return 1
    else: return 0

In [4]:
df['stockPriceChange'] = df.apply(stockDevelopement, axis=1)

In [5]:
df = df[['id','headline', 'stock', 'headline_cleaned','open','close', "stockPriceChange"]]
df = df.rename(columns={'headline': '_headline', 'headline_cleaned':'_headline_cleaned', 'stock': '_stock', 'open': '_open', 'close':'_close', 'stockPrice_Change':'_stockPrice_Change'})

In [6]:
def tokenz_in_sentence(hl):
    word_list = hl[1:-1].split(", ")
    word_list = [word[1:-6] for word in word_list]
    sentence = ' '.join(word_list)
    return sentence

In [7]:
df['_headline_sentence'] = df._headline_cleaned.apply(tokenz_in_sentence)

In [8]:
df = df.set_index('id')

In [9]:
# df['stockPriceChange'] = df['stockPriceChange'].apply(lambda value: value*100)
# df['stockPriceChange'] = df['stockPriceChange'].astype(np.float16)
# df['_open'] = df['_open'].astype(np.float16)
# df['_close'] = df['_close'].astype(np.float16)

In [10]:
df_set = df['_headline_sentence'].tolist()

In [11]:
df.shape

(161478, 7)

In [12]:
count_vectorizer = CountVectorizer(analyzer="word", stop_words="english")
tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')

count_wm = count_vectorizer.fit_transform(df_set)
tfidf_wm = tfidf_vectorizer.fit_transform(df_set)

count_tokenz = count_vectorizer.get_feature_names()
tfidf_tokenz = tfidf_vectorizer.get_feature_names()

df_countvect = pd.DataFrame(data=count_wm.toarray(), index=df.index, columns=count_tokenz)
df_tfidfvect = pd.DataFrame(data= tfidf_wm.toarray(), index=df.index, columns=tfidf_tokenz)

In [13]:
cols_to_delete = []
for column in df_tfidfvect:
    count = df_tfidfvect[df_tfidfvect[column] > 0].shape[0]
    if count<20:
        cols_to_delete.append(column)

print(len(cols_to_delete))
print(df_tfidfvect.shape[1])

In [14]:
df_tfidfvect.drop(axis=1, columns=cols_to_delete, inplace=True)

In [15]:
mergred = df.merge(df_tfidfvect, on='id')

In [16]:
mergred.head()

,_headline,_stock,_headline_cleaned,_open,_close,stockPriceChange,_headline_sentence,131,aare,aaron,...,zebra,zebu,zero,zeus,zhou,ziegler,zinc,zion,zip_up,zone
id,,,,,,,,,,,,,,,,,,,,,
0,Stocks That Hit 52-Week Highs On Friday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'frida...",92.13,90.38,1,stock hit high friday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stocks That Hit 52-Week Highs On Wednesday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'wedne...",90.65,90.49,-1,stock hit high wednesday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,71 Biggest Movers From Friday,A,"['large.a.01', 'mover.n.01', 'friday.n.01']",86.23,86.13,-1,large mover friday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,46 Stocks Moving In Friday's Mid-Day Session,A,"['stock.n.01', 'travel.v.01', 'friday.n.01', '...",85.00,84.98,-1,stock travel friday session,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B of A Securities Maintains Neutral on Agilent...,A,"['security.n.01', 'neutral.n.01', 'technology....",85.00,84.98,-1,security neutral technology raise monetary_val...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
import numpy as np

In [18]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 30, stop = 100, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 50, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 3]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [30, 37, 45, 53, 61, 68, 76, 84, 92, 100], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, None], 'min_samples_split': [2, 3], 'min_samples_leaf': [1, 2], 'bootstrap': [True, False]}


In [19]:
from sklearn.ensemble import RandomForestRegressor


In [20]:
# rf = RandomForestRegressor()
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2)

In [21]:
X = mergred.drop(columns=['_headline', '_stock', '_headline_cleaned', '_open', '_close', 'stockPriceChange', '_headline_sentence'])
y= mergred['stockPriceChange']

In [22]:
# Fit the random search model
# rf_random.fit(X, y)


In [23]:
# rf_random.best_params_

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=0)

MemoryError: Unable to allocate 8.39 GiB for an array with shape (8713, 129182) and data type float64

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor( n_estimators=37, min_samples_split=2, min_samples_leaf=2, max_features='sqrt', max_depth=10, bootstrap=True)
regressor.fit(X_train, y_train,)
y_pred = regressor.predict(X_test)

In [ ]:
y_pred_bin = []
for value in y_pred:
    if value < 1:
        y_pred_bin.append(-1)
    elif value > 1:
        y_pred_bin.append(1)
    else: y_pred_bin.append(0)

In [ ]:
from sklearn import metrics
import numpy as np

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_bin))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_bin))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_bin)))

Mean Absolute Error: 0.3422405251424325
Mean Squared Error: 0.6830257617042358
Root Mean Squared Error: 0.8264537262933938


In [ ]:
print('Accuracy Score:', metrics.accuracy_score(y_test, y_pred_bin))

Accuracy Score: 0.8281520931384692
